# Installation
```
git clone git@github.com:vllm-project/llm-compressor.git\
cd llm-compressor\
micromamba create -n weight-analyzer python=3.11\
pip install -e .

```

In [3]:
import transformers
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from llmcompressor.transformers import SparseAutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype='auto',
    cache_dir="/nm/drive0/shashata/weight-analysis"
)


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

In [ ]:
model